In [1]:
#coding: utf-8
import pandas
import nltk

__author__ = "Ionesio Junior"

# Color consts
OKGREEN = '\033[92m'
FAIL = '\033[91m'
ENDC = '\033[0m'

match_words = {}

### Construindo o dicionário de pesquisa
Nosso dicionário de Pesquisa é construído basicamente por 2 métodos.
    - extract_words (pega sequencialmente os textos da tabela e separa por palavra associando-as ao ID da notícia)
    - add_dictionary (preenche o dicionário com os id's(valor) que possuem aquela determinada palavra(chave))

In [2]:
def add_dictionary( info_tuple ):
    '''
        This method fill lists of news index with match words

        Args:
            info_tuple(String, Int) : this tuple contain a specific word and index of news
    '''
    word, index = info_tuple
    try:
        match_words[ word.lower() ].add(index)
    except KeyError:
        match_words[ word.lower() ] = set([ index ])

In [3]:
def extract_words( text , id_col ):
    ''' 
        This method extract match words from all text tables and store in a dictionary correlating with news index
        
        Args:
                text() : table with text to extract words
                id_col : table with news index
    '''
    [ map( add_dictionary, [ ( word, id_col[i] ) for word in nltk.word_tokenize( text[i] ) ] ) for i in xrange( len(text) ) ]

### Buscando dados
Com esse método, podemos fazer buscas de múltiplas palavras usando(ou não) operadores AND/OR para restringir/abranger nossos resultados 

In [4]:
def search( words ):
    '''
        This method search a set of words using AND/OR operators to guide how to search

        Args:
                words(String) : string with words to be searched (Ex: word1 AND word2 AND ... | word1 OR word2 OR ... )
        Return:
                index_list[Int] : list of news index that matches with words
    '''
    if ( "AND" in words ):
        list_of_words = map( lambda x: x.strip(), words.split("AND") )
        return reduce( lambda x,y : list( x & y ) , [ match_words[word.lower()] for word in list_of_words ]      )
    elif ("OR" in words ):
        list_of_words = map(lambda x: x.strip(), words.split("OR"))
        return reduce( lambda x,y : list( x | y ), [ match_words[word.lower()] for word in list_of_words ] )
    else:
        return list(match_words[words])

### Testando o algoritmo
Para testar se o algoritmo esta funcionando como deveria, temos um método com alguns casos para comparar o resultado esperado em relação ao recebido.

In [5]:
def debug_test(text, expected_value):
	result =  len(search(text)) == expected_value
	if result == True:
		print "Testing \"" + text + "\"... " + OKGREEN + "Sucess!!" + ENDC
	else:
		print "Testing \"" + text + "\"... " + FAIL + "Fail!!" + ENDC
    

def test_search():
    #OR
    debug_test("debate OR presidencial",1770)
    debug_test("presidenciáveis OR corruptos", 164)
    debug_test("Belo OR Horizonte", 331)

    #AND
    debug_test("Belo AND Horizonte", 242)
    debug_test("presidenciáveis AND corruptos", 0)
    debug_test("debate AND presidencial",201)
    debug_test("Campina AND Grande",12)

### Rodando
Por fim, juntamos todas as peças e testamos para validar se tudo está funcionando da forma correta 

In [6]:
if __name__ == "__main__":
    file_csv = pandas.read_csv("noticias_estadao.csv")
    text = file_csv.titulo + " " + file_csv.conteudo
    extract_words(text,file_csv.idNoticia)
    test_search()

Testing "debate OR presidencial"... Sucess!!
Testing "presidenciáveis OR corruptos"... Sucess!!
Testing "Belo OR Horizonte"... Sucess!!
Testing "Belo AND Horizonte"... Sucess!!
Testing "presidenciáveis AND corruptos"... Sucess!!
Testing "debate AND presidencial"... Sucess!!
Testing "Campina AND Grande"... Sucess!!
